<a href="https://colab.research.google.com/github/bingsen-wang/EE-Fundamentals/blob/main/BodePlot_LeadOrLagCompensator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center><h1><b>Lead or Lag Compensator</h1></b>

Dr. Bingsen Wang

7/4/2025
</center>

#Python Code

In [ ]:
from matplotlib import pyplot,animation,cm
import numpy as np
pyplot.rcParams.update({"font.family":"sans-serif", "mathtext.fontset":"cm"})

def T_lead(s,wz,wp):
  return (1+s/wz)/(1+s/wp)
def dB(x):
  return 20*np.log10(np.abs(x))
def ph(x):
  return np.angle(x,deg=True)

Nf=480
wz=np.logspace(-1,1,Nf); wp=np.logspace(1,-1,Nf)
omega=np.logspace(-2.5,2.5,501)
cl = list(cm.tab10.colors) #colors of lines
fig = pyplot.figure(figsize=([9,9]))
ax1 = fig.add_axes([.1,.5,.875,.4],xlim=[min(omega),max(omega)],ylim=[-43,43],fc='none',xscale='log');#ax1.axis('off')
ax2 = fig.add_axes([.1,.075,.875,.4],xlim=[min(omega),max(omega)],ylim=[-95,95],fc='none',xscale='log');#ax2.axis('off')
ax = fig.add_axes([0,0,1,1],xlim=[-1.2,1.2],ylim=[-1.2,1.25],fc='none');ax.axis('off')
ax1.set_yticks(np.linspace(-40,40,9))
ax1.set_xticklabels('');
ax1.grid(visible=True,which='major',axis='x',lw=.5,color='.5')
ax1.grid(visible=True,which='major',axis='y',lw=.5,color='.5')
ax1.grid(visible=True,which='minor',axis='x',lw=.5,color='.5',alpha=.5)
ax1.set_ylabel(r'$\text{Gain (dB)}$',size=20,loc='center',labelpad=0)

ax2.set_yticks(np.linspace(-90,90,13))
ax2.grid(visible=True,which='major',axis='x',lw=.5,color='.5')
ax2.grid(visible=True,which='major',axis='y',lw=.5,color='.5')
ax2.grid(visible=True,which='minor',axis='x',lw=.5,color='.5',alpha=.5)
ax2.set_xlabel(r'$\omega\ \text{(rad/s)}$  ',size=25,loc='right',labelpad=-10)
ax2.set_ylabel(r'$\text{Phase (deg)}$',size=20,loc='center',labelpad=0)
txts=[ax.text(0,1.1,"Bode Plots of Lead Compensator",size=32,ha='center')]

ax.text(-.85,.9,r'$T(s) = \frac{1+\frac{s}{\omega_z}}{1+\frac{s}{\omega_p}}$',
        size=28,ha='left',va='top',multialignment='left',
        bbox=dict(boxstyle='roundtooth',pad=.8,facecolor='w',edgecolor=cl[2],lw=5))
txts+=[ax2.text(0,0,'',size=20,c='k',rotation=90,va='center',ha=['right','left'][k]) for k in range(2)] #omega_z, omega_p

lines=[ax1.plot([],[],lw=4-2*k,c=cl[k],ls=['-','--'][k])[0] for k in range(2)]
lines+=[ax1.plot([],[],lw=1.5,c='k',ls='--')[0] for _ in range(2)]
lines+=[ax2.plot([],[],lw=4-2*k,c=cl[k],ls=['-','--'][k])[0] for k in range(2)]
lines+=[ax2.plot([],[],lw=1.5,c='k',ls='--')[0] for _ in range(2)]
def animate(i):
  if i%50==0: print(i)
  if i==Nf//2:
    txts[0].set_text('Bode Plots of Lag Compensator')
    txts[1].set_ha('left')
    txts[2].set_ha('right')
  TdB=dB(T_lead(1j*omega,wz[i],wp[i]))
  TdB_appr=np.maximum(0*omega,dB(1j*omega/wz[i]))-np.maximum(0*omega,dB(1j*omega/wp[i]))
  TdB_appr_z=np.maximum(0,dB(1j))-np.maximum(0,dB(1j*wz[i]/wp[i]))
  TdB_appr_p=np.maximum(0,dB(1j*wp[i]/wz[i]))-np.maximum(0,dB(1j))
  Phase=ph(T_lead(1j*omega,wz[i],wp[i]))
  [lines[k].set_data(omega,[TdB,TdB_appr][k]) for k in range(2)]
  [lines[k+2].set_data([[wz[i],wp[i]][k]]*2,[-43,[TdB_appr_z,TdB_appr_p][k]]) for k in range(2)]

  lines[4].set_data(omega,Phase)
  [lines[k+6].set_data([[wz[i],wp[i]][k]]*2,[-95,95]) for k in range(2)]
  [txts[k+1].set_position([[wz[i],wp[i]][k],-45+(i//(Nf//2))*90]) for k in range(2)]

  [txts[k+1].set_text([rf'$\omega_z={wz[i]:.2f}$',rf'$\omega_p={wp[i]:.2f}$'][k]) for k in range(2)]

  return
ax.text(np.average(ax.get_xlim()),ax.get_ylim()[0]*.99+ax.get_ylim()[1]*.01,r'@eeanimation',size=12,c='.2',alpha=.3,ha='center',va='bottom')
anim = animation.FuncAnimation(fig, animate, frames=Nf, interval=20)
anim.save("BodePlot_leadlag.mp4",writer=animation.FFMpegWriter(fps=60),dpi=200)